In [49]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
pal = sns.color_palette()
import pickle
from sklearn.svm import SVR
from sklearn.metrics import log_loss


from text_processor import TextProcessor

In [17]:
tp = TextProcessor()

## Load Doc2Vec

In [3]:
model = Doc2Vec.load('models/doc2vec')

In [7]:
model.wv.most_similar(['woman'], topn=10)

[('man', 0.9261379837989807),
 ('girl', 0.8550839424133301),
 ('guy', 0.8494753837585449),
 ('boy', 0.8471918106079102),
 ('person', 0.8208020925521851),
 ('women', 0.7791064381599426),
 ('men', 0.7236971259117126),
 ('girlfriend', 0.7207836508750916),
 ('wife', 0.7048382759094238),
 ('boyfriend', 0.7022792100906372)]

## Load Data

In [39]:
df_train = pd.read_csv('data/train.csv')
df_train = df_train.dropna() 
print("The training dataset has shape {}".format(df_train.shape))
df_train.head()

The training dataset has shape (404287, 6)


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [41]:
target = 'is_duplicate'
features = ['question1', 'question2']

X = df_train[features].as_matrix()
y = df_train[target].as_matrix()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Convert Data

In [42]:
def row_to_vec(row):
    """
    Return a [1, 2 * n_features] vector which is the concatenation of the word2vec vectors for the 
    two sentences (average of words)
    """
#     q1 = tp.clean(row[0])
#     q2 = tp.clean(row[1])
    q1 = tp.tokenize(row[0])
    q2 = tp.tokenize(row[1])
    
    q1_vec = np.mean(np.array([model.infer_vector([word]) for word in q1]), axis=0)
    q2_vec = np.mean(np.array([model.infer_vector([word]) for word in q2]), axis=0)
    
    
    return q1_vec + q2_vec

In [43]:
%%time
X_vec = np.apply_along_axis(row_to_vec, 1, X)

CPU times: user 23min 39s, sys: 11.5 s, total: 23min 51s
Wall time: 26min 30s


In [44]:
pickle.dump( {'X_vec': X_vec, 'y_vec': y}, open( "data/vec.dump", "wb" ) )

## SVM Classifier

In [45]:
y_vec = y

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X_vec, y_vec, test_size=0.2, random_state=42)

In [48]:
clf = SVR(C=1.0, epsilon=0.2)
clf.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [50]:
y_pred = clf.predict(X_test)

In [51]:
log_loss(y_test, y_pred)

0.6986277499684297